In [1]:
import numpy as np
import random
from os.path import abspath, join

In [2]:
class Network:

    def __init__(self, sizes: list):
        """ 
        sizes 包含各層神經元的數量, 假設建構一個2層的NN, 
        則目標會含有:input layer x1, hidden layer x1, output layer x1
        若每層的神經元數量分別是784,15,1則創建時指定list object [784,15,1]即可。

        example: net = Network([784, 15, 1]) 
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.train_acc = 0
        self.test_acc = 0
        self.epoch = 0
        self.eta = 0

    def feedforward2(self, a):
        """Return the output of the network if ``a`` is input."""
        zs = []
        activations = [a]

        activation = a
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        activations.append(softmax(zs[-1]))
        activations.pop(-2)

        return (zs, activations)

    def update_mini_batch2(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        batch_size = len(mini_batch)

        # transform to (input x batch_size) matrix
        x = np.asarray([_x.ravel() for _x, _y in mini_batch]).transpose()
        # transform to (output x batch_size) matrix
        y = np.asarray([_y.ravel() for _x, _y in mini_batch]).transpose()

        nabla_b, nabla_w = self.backprop2(x, y)
        self.weights = [w - (eta / batch_size) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / batch_size) * nb for b, nb in zip(self.biases, nabla_b)]

        return

    def backprop2(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [0 for i in self.biases]
        nabla_w = [0 for i in self.weights]

        # feedforward
        zs, activations = self.feedforward2(x)

        # backward pass
        delta = self.cost_derivative(activations[-1], y) * softmax(zs[-1], prime=True)
        nabla_b[-1] = delta.sum(1).reshape([len(delta), 1]) # reshape to (n x 1) matrix
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid(z, prime=True)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta.sum(1).reshape([len(delta), 1]) # reshape to (n x 1) matrix
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())

        return (nabla_b, nabla_w)

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        eva_train_result = []
        eva_test_result = []
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            #self.update_mini_batch2(mini_batches, eta)
            for mini_batch in mini_batches:
                self.update_mini_batch2(mini_batch, eta)
            eta *= 0.98
            self.epoch = epoch
            self.eta = eta
            if test_data:
                eva_train_result.append(self.evaluate_train(training_data))
                eva_test_result.append(self.evaluate(test_data))
                print(f"Epoch {epoch}:bias {eva_train_result[-1]} / {n} ({eva_train_result[-1]/n*100}%)")
                print(f"Epoch {epoch}:validation {eva_test_result[-1]} / {n_test} ({eva_test_result[-1]/n_test*100}%)")
                try:
                    overfitting = eva_train_result[-2]/n > eva_train_result[-1]/n and \
                        eva_test_result[-2]/n_test < eva_test_result[-1]/n_test
                except:
                    overfitting = False
                if overfitting and epoch > 5:
                    print(f'bias raise up but validation down, stop training to avoid overfitting.')
                    self.train_acc = eva_train_result[-2]/n*100
                    self.test_acc = eva_test_result[-2]/n_test*100
                    break
                self.train_acc = eva_train_result[-1]/n*100
                self.test_acc = eva_test_result[-1]/n_test*100
            else:
                print(f"Epoch {epoch} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid(zs[-1], prime=True)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid(z, prime=True)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward2(x)[1][-1]), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def evaluate_train(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward2(x)[1][-1]), np.argmax(y))
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [3]:
#### Miscellaneous functions
def sigmoid(z, prime=False):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z)) if not prime else sigmoid(z)*(1.0-sigmoid(z))

def softmax(z , prime=False):
    # Numerically stable with large exponentials
    exps = np.exp(z - z.max())
    if prime: return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
    return exps / np.sum(exps, axis=0)

In [4]:
import mnist_loader
training_data2, validation_data2, test_data2 = \
    mnist_loader.load_data_wrapper()

In [11]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=validation_data)

Epoch 0: 9059 / 10000
Epoch 1: 9269 / 10000
Epoch 2: 9340 / 10000
Epoch 3: 9361 / 10000
Epoch 4: 9419 / 10000
Epoch 5: 9408 / 10000
Epoch 6: 9426 / 10000
Epoch 7: 9429 / 10000
Epoch 8: 9443 / 10000
Epoch 9: 9453 / 10000
Epoch 10: 9464 / 10000
Epoch 11: 9448 / 10000
Epoch 12: 9488 / 10000
Epoch 13: 9502 / 10000
Epoch 14: 9484 / 10000
Epoch 15: 9439 / 10000
Epoch 16: 9472 / 10000
Epoch 17: 9488 / 10000
Epoch 18: 9474 / 10000
Epoch 19: 9472 / 10000
Epoch 20: 9468 / 10000
Epoch 21: 9478 / 10000
Epoch 22: 9476 / 10000
Epoch 23: 9479 / 10000
Epoch 24: 9496 / 10000
Epoch 25: 9490 / 10000
Epoch 26: 9492 / 10000
Epoch 27: 9488 / 10000
Epoch 28: 9484 / 10000
Epoch 29: 9485 / 10000


In [19]:
class Loader:
    """define function for loading txt file"""

    @staticmethod
    def load_img(path, data=[]):
        """load train_img/test_img txt file"""
        try:
            with open(f"{abspath('.')}{path}", 'r') as txtFile:
                for line in txtFile:
                    line = line.replace('\n', '').split(',')
                    x = [[float(_)] for _ in line]
                    data.append(np.array(x))
            return data
        except Exception as e:
            raise

    @staticmethod
    def load_label(path, one_hot_vector=False, data=[]):
        """load train_label/test_label txt file"""
        try:
            with open(f"{abspath('.')}{path}", 'r') as txtFile:
                for line in txtFile:
                    x = line.replace('\n', '').split(',')
                    data.append(np.int64(x[0]))
            return data
        except Exception as e:
            raise

    @staticmethod
    def one_hot_vector(label, data=[]):
        for x in label:
            if x == 0:
                x = [[1.], [0.], [0.]]
            elif x == 1:
                x = [[0.], [1.], [0.]]
            elif x == 2:
                x = [[0.], [0.], [1.]]
            data.append(np.array(x))
        return data

    def load_data_wrapper(self):
        # load file
        tr_d = self.load_img('/data/train_img.txt')
        tr_l = self.load_label('/data/train_label.txt')
        td_d = self.load_img('/data/test_img.txt')

        # grouping
        training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[:6000]]
        training_data = list(zip(training_inputs, fi1e.one_hot_vector(tr_l[:6000])))

        validation_inputs = [np.reshape(x, (784, 1)) for x in tr_d[6000:]]
        validation_data = list(zip(validation_inputs, tr_l[6000:]))

        test_data = list(zip(td_d))

        return training_data, validation_data, test_data

In [20]:
fi1e = Loader()
training_data, validation_data, test_data = fi1e.load_data_wrapper()

In [21]:
from os.path import abspath, join
fi1e = Loader()
tr_d = fi1e.load_img('/data/train_img.txt')
tr_l = fi1e.load_label('/data/train_label.txt')
td_d = fi1e.load_img('/data/test_img.txt')

In [13]:
# normalization & grouping
training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[:6000]]
training_data = list(zip(training_inputs, fi1e.one_hot_vector(tr_l[:6000])))

validation_inputs = [np.reshape(x, (784, 1)) for x in tr_d[6000:]]
validation_data = list(zip(validation_inputs, tr_l[6000:]))

test_data = list(zip(td_d))


NameError: name 'tr_d' is not defined

In [24]:
aa = []
with open(f"./data/test_img.txt", 'r') as txtFile:
    for f in txtFile:
        f = f.replace('\n', '').split(',')
        x = [[float(_)] for _ in f]
        aa.append(np.array(x))
print(len(aa))

2000


In [11]:
net2 = Network([784, 30, 10])

In [12]:
net2.SGD(training_data2, 15, 10, 3, test_data=validation_data2)

NameError: name 'training_data2' is not defined

In [11]:
nn_structure = [784, 128, 64, 3]
net = Network(nn_structure)
net.SGD(training_data, 15, 32, 1, test_data=validation_data)
print(f'Final bias: {net.train_acc}%')
print(f'Final validation: {net.test_acc}%')

Epoch 0:bias 5394 / 6000 (89.9%)
Epoch 0:validation 1751 / 2000 (87.55%)
Epoch 1:bias 5494 / 6000 (91.56666666666666%)
Epoch 1:validation 1829 / 2000 (91.45%)
Epoch 2:bias 5611 / 6000 (93.51666666666667%)
Epoch 2:validation 1854 / 2000 (92.7%)
Epoch 3:bias 5626 / 6000 (93.76666666666667%)
Epoch 3:validation 1853 / 2000 (92.65%)
Epoch 4:bias 5688 / 6000 (94.8%)
Epoch 4:validation 1871 / 2000 (93.55%)
Epoch 5:bias 5733 / 6000 (95.55%)
Epoch 5:validation 1907 / 2000 (95.35%)
Epoch 6:bias 5688 / 6000 (94.8%)
Epoch 6:validation 1878 / 2000 (93.89999999999999%)
Epoch 7:bias 5744 / 6000 (95.73333333333333%)
Epoch 7:validation 1909 / 2000 (95.45%)
Epoch 8:bias 5748 / 6000 (95.8%)
Epoch 8:validation 1893 / 2000 (94.65%)
Epoch 9:bias 5787 / 6000 (96.45%)
Epoch 9:validation 1911 / 2000 (95.55%)
Epoch 10:bias 5616 / 6000 (93.60000000000001%)
Epoch 10:validation 1850 / 2000 (92.5%)
Epoch 11:bias 5777 / 6000 (96.28333333333333%)
Epoch 11:validation 1900 / 2000 (95.0%)
Epoch 12:bias 5731 / 6000 (95.5

In [79]:
print(f'###### Details ######')
print(f'1.Data Count')
print(f' -  Trainning data: {len(training_data)}')
print(f' - Validation data: {len(validation_data)}')
#print(f' -    Predict data: {len(unknown_X)}')
#print(f' -           Total: {len(train_X)+len(validation_X)+len(unknown_X)}')
print(f'')
print(f'2.Hidden layer')
print(f' -     Hidden Layer count: {len(nn_structure)-2}')
for cnt in range(1,len(nn_structure)-1):
    print(f' - Layer#{cnt}\'s Neuron count: {nn_structure[cnt]}')
print(f'')
print(f'3.Final epoch: {net.epoch}')
print(f'4.Final learning rate: {net.eta}%')
print(f'5.Accuracy Rate')
print(f' -       Bias: {net.train_acc}%')
print(f' - Validation: {net.test_acc}%')
print(f'')
print(f'6.Error Rate')
print(f' -     Bias: {round(100-net.train_acc, 2)}%')
print(f' - Variance: {round(-net.test_acc+net.train_acc, 2)}%')
print(f'')
print(f'>> Process End-Up !')

###### Details ######
1.Data Count
 -  Trainning data: 6000
 - Validation data: 2000

2.Hidden layer
 -     Hidden Layer count: 2
 - Layer#1's Neuron count: 128
 - Layer#2's Neuron count: 64

3.Final epoch: 8
4.Final learning rate: 0.8337477621301497%
5.Accuracy Rate
 -       Bias: 96.48333333333333%
 - Validation: 96.2%

6.Error Rate
 -     Bias: 3.52%
 - Variance: 0.28%

>> Process End-Up !


In [26]:
def vvv():
    return 10,5
vvv()[1]

5